In [15]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, Model
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.optimizers import Adam
from PIL import Image

# Define paths
train_image_dir = '/WAVE/projects/CSEN-342-Wi24/data/pr2/train/images'
train_label_file = '/WAVE/projects/CSEN-342-Wi24/data/pr2/train/labels.txt'
val_image_dir = '/WAVE/projects/CSEN-342-Wi24/data/pr2/val/images'
val_label_file = '/WAVE/projects/CSEN-342-Wi24/data/pr2/val/labels.txt'
test_image_dir = '/WAVE/projects/CSEN-342-Wi24/data/pr2/test/images'
output_file = 'predictions.txt'

# Define class mappings
class_mapping = {'1': 'car', '2': 'medium truck', '3': 'large truck'}

# Define image size
image_size = (416, 416)

# Function to parse label file
def parse_label(label_file):
    labels = {}
    with open(label_file, 'r') as file:
        lines = file.readlines()
        for line in lines:
            parts = line.strip().split()
            image_id = parts[0]
            label = parts[1]
            cx = int(parts[2])
            cy = int(parts[3])
            w = int(parts[4])
            h = int(parts[5])
            labels.setdefault(image_id, []).append((label, cx, cy, w, h))
    return labels

# Parse labels
train_labels = parse_label(train_label_file)
val_labels = parse_label(val_label_file)

# Load pre-trained MobileNetV2 model without top classification layers
base_model = MobileNetV2(input_shape=(image_size[0], image_size[1], 3),
                         include_top=False,
                         weights='imagenet')

# Freeze the pre-trained layers
base_model.trainable = False

# Add custom classification layers
x = base_model.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(512, activation='relu')(x)
x = layers.Dropout(0.5)(x)
predictions = layers.Dense(3, activation='softmax')(x)

# Create model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile model
model.compile(optimizer=Adam(lr=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Define a function to load and preprocess images
def load_and_preprocess_image(image_path):
    img = Image.open(image_path)
    img = img.resize(image_size)
    img_array = np.array(img) / 255.0
    return img_array

# Define a function to generate predictions
def generate_predictions(image_dir, labels, output_file):
    with open(output_file, 'w') as file:
        for image_id, image_labels in labels.items():
            image_id_padded = image_id.zfill(5)  # Padding image_id with zeros
            image_path = os.path.join(image_dir, image_id_padded + '.jpeg')  # Updated image path
            image = load_and_preprocess_image(image_path)
            detections = model.predict(np.expand_dims(image, axis=0))
            for i, detection in enumerate(detections):
                label, cx, cy, w, h = image_labels[i]
                conf = detection[int(label) - 1]  # Confidence score for the predicted class
                cx_norm = cx / image.shape[1]  # Normalize x-coordinate
                cy_norm = cy / image.shape[0]  # Normalize y-coordinate
                w_norm = w / image.shape[1]   # Normalize width
                h_norm = h / image.shape[0]   # Normalize height
                cx_norm = np.clip(cx_norm, 0, 1)  # Clip x-coordinate to [0, 1]
                cy_norm = np.clip(cy_norm, 0, 1)  # Clip y-coordinate to [0, 1]
                w_norm = np.clip(w_norm, 0, 1)    # Clip width to [0, 1]
                h_norm = np.clip(h_norm, 0, 1)    # Clip height to [0, 1]
                conf = np.clip(conf, 0, 1)  # Clip confidence score to [0, 1]
                file.write(f"{image_id} {label} {cx_norm} {cy_norm} {w_norm} {h_norm} {conf}\n")
            print(f'Processed {image_id}/{2000} images', end='\r')


# Train the model
# You need to prepare your training data, convert labels to one-hot encoding, etc.

# Generate predictions for validation set
generate_predictions(val_image_dir, val_labels, output_file)


1/1 [==============================] - 0s 41ms/step
